In [ ]:
from CSVModifier import aggregate_high_freq_to_low, add_factors, clean_outliers
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd


from Util import fetch_stock_data_finnhub_paginated

In [ ]:
SYMBOL = 'SPY'
START_DATE = '2020-01-01'
END_DATE = '2025-05-03'
FINHUB_TOKEN = 'cvop3lhr01qihjtq3uvgcvop3lhr01qihjtq3v00'
HIFRREQ = '15'
LOFREQ = '60'

In [ ]:
# 请求数据
df_hiFreq = fetch_stock_data_finnhub_paginated(
    symbol= SYMBOL,
    start_date=START_DATE,
    end_date=END_DATE,
    interval=HIFRREQ, # 单位为分钟
    token=FINHUB_TOKEN,
    chunk_days=25,
    verbose=True
)

df_loFreq = fetch_stock_data_finnhub_paginated(
    symbol= SYMBOL,
    start_date=START_DATE,
    end_date=END_DATE,
    interval=LOFREQ, # 单位为分钟
    token=FINHUB_TOKEN,
    chunk_days=25,
    verbose=True
)


In [ ]:
# 检查数据
df_loFreq.tail(5)
# df_hiFreq.tail(5)

In [ ]:
# 把高频转换成低频集合
df_hiFreq_agg = aggregate_high_freq_to_low (df_hiFreq)

# 计算因子
df_loFreq_factored = add_factors (df_loFreq)

# 拼接
df_merged = pd.merge (df_loFreq_factored, df_hiFreq_agg, how='left', on='timestamp')
df_merged = clean_outliers(df_merged, z_thresh=10, show_msg=True)

In [ ]:
# 检查数值
df_merged[df_merged['close_volatility'] == 0]

In [ ]:
import re
path = f"{SYMBOL}_{START_DATE}_{END_DATE}_{LOFREQ}m.csv"
# path = re.sub(r'[^\w\-]', '', path)
df_merged.to_csv(path, index=False)